In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 3
# Number of dimensions of each input
D = 4
# Create an empty list
all_x = []
# Create elements x_n and append to list
for n in range(N):
  all_x.append(np.random.normal(size=(D,1)))
# Print out the list
print(all_x)

[array([[ 1.78862847],
       [ 0.43650985],
       [ 0.09649747],
       [-1.8634927 ]]), array([[-0.2773882 ],
       [-0.35475898],
       [-0.08274148],
       [-0.62700068]]), array([[-0.04381817],
       [-0.47721803],
       [-1.31386475],
       [ 0.88462238]])]


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [7]:
# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters - 초기화
omega_q = np.random.normal(size=(D,D))
omega_k = np.random.normal(size=(D,D))
omega_v = np.random.normal(size=(D,D))
beta_q = np.random.normal(size=(D,1))
beta_k = np.random.normal(size=(D,1))
beta_v = np.random.normal(size=(D,1))

Now let's compute the queries, keys, and values for each input

In [9]:
# Make three lists to store queries, keys, and values
all_queries = []
all_keys = []
all_values = []
# For every input
for x in all_x:
  # TODO -- compute the keys, queries and values.
  # Replace these three lines
  query = omega_q@x+beta_q
  key = omega_k@x+beta_k
  value = omega_v@x+beta_v

  all_queries.append(query)
  all_keys.append(key)
  all_values.append(value)

We'll need a softmax function (equation 12.5) -- here, it will take a list of arbitrary numbers and return a list where the elements are non-negative and sum to one

In [10]:
def softmax(items_in):

  # TODO Compute the elements of items_out
  # Replace this line
  items_out = np.exp(items_in)/sum(np.exp(items_in))

  return items_out

Now compute the self attention values:

In [28]:
all_values[0]*

array([[-0.11861745],
       [ 3.2262959 ],
       [-3.79442242],
       [-1.28263336]])

In [29]:
# Create emptymlist for output
all_x_prime = []

# For each output
for n in range(N):
  # Create list for dot products of query N with all keys
  all_km_qn = []
  # Compute the dot products
  for key in all_keys:
    # TODO -- compute the appropriate dot product
    # Replace this line - query*key dot product
    dot_product = np.transpose(all_queries[n]).dot(key)

    # Store dot product
    all_km_qn.append(dot_product)

  # Compute dot product
  attention = softmax(all_km_qn)
  # Print result (should be positive sum to one)
  print("Attentions for output ", n)
  print(attention)

  # TODO: Compute a weighted sum of all of the values according to the attention
  # (equation 12.3)
  # Replace this line
  x_prime = all_values[0]*attention[0]+all_values[1]*attention[1]+all_values[2]*attention[2]

  all_x_prime.append(x_prime)

# Print out true values to check you have it correct
print("x_prime_0_calculated:", all_x_prime[0].transpose())
print("x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]")
print("x_prime_1_calculated:", all_x_prime[1].transpose())
print("x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]")
print("x_prime_2_calculated:", all_x_prime[2].transpose())
print("x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]")


Attentions for output  0
[[[1.24326146e-13]]

 [[9.98281489e-01]]

 [[1.71851130e-03]]]
Attentions for output  1
[[[2.79525306e-12]]

 [[5.85506360e-03]]

 [[9.94144936e-01]]]
Attentions for output  2
[[[0.00505708]]

 [[0.00654776]]

 [[0.98839516]]]
x_prime_0_calculated: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_1_calculated: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_2_calculated: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]
x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]


Now let's compute the same thing, but using matrix calculations. We'll store the N inputs $x_n \in R^{D}$
 in the columns of a $D \times N$
 matrix, using equations 12.6 and 12.7/8. \n

Note: The book uses column vectors (for compatibility with the rest of the text), but in the wider literature it is more normal to store the inputs in the rows of a matrix; in this case, the computation is the same, but all the matrices are transposed and the operations proceed in the reverse order.

In [30]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in)
  # Sum over columns
  denom = np.sum(exp_values, axis = 0)
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax

In [82]:
 # Now let's compute self attention in matrix form
def self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):

  # TODO -- Write this function
  # 1. Compute queries, keys, and values
  queries = beta_q+omega_q@X
  keys = beta_k+omega_k@X
  values = beta_v+omega_v@X

  # 2. Compute dot products
  all_km_qn = np.transpose(keys)@queries

  # 3. Apply softmax to calculate attentions
  attention = softmax_cols(all_km_qn)

  # 4. Weight values by attentions
  X_prime = values@attention

  return np.transpose(X_prime)

In [83]:
# Copy data into matrix
X = np.zeros((D, N))
X[:,0] = np.squeeze(all_x[0])
X[:,1] = np.squeeze(all_x[1])
X[:,2] = np.squeeze(all_x[2])

# Run the self attention mechanism
X_prime = self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]
 [ 1.64201168 -0.08470004  4.02764044  2.18690791]
 [ 1.61949281 -0.06641533  3.96863308  2.15858316]]


If you did this correctly, the values should be the same as above.

TODO:

Print out the attention matrix You will see that the values are quite extreme (one is very close to one and the others are very close to zero. Now we'll fix this problem by using scaled dot-product attention.

In [90]:
# Now let's compute self attention in matrix form
def scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):

  # TODO -- Write this function
  # 1. Compute queries, keys, and values
  queries = beta_q+omega_q@X
  keys = beta_k+omega_k@X
  values = beta_v+omega_v@X

  # 2. Compute dot products
  all_km_qn = np.transpose(keys)@queries

  # 3. Scale the dot products as in equation 12.9 - 근데 차원 수가 같으므로 그냥 전체적으로 2 나눠준 꼴이 된다.
  all_km_qn/2

  # 4. Apply softmax to calculate attentions
  attention = softmax_cols(all_km_qn)
  
  # 5. Weight values by attentions
  # Replace this line
  X_prime = values@attention

  return np.transpose(X_prime)

In [91]:
# Run the self attention mechanism
X_prime = scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]
 [ 1.64201168 -0.08470004  4.02764044  2.18690791]
 [ 1.61949281 -0.06641533  3.96863308  2.15858316]]
